# SparkContext - number of workers and lazy evaluation

## Checking the impact of number of workers
While initializing the `SparkContext`, we can specify number of worker nodes. Generally, it is recommended to have one worker per core of the machine. But it can be smaller or larger. In the following code, we will examine the impact of number of worker cores on some parallelized operation.

In [1]:
from time import time
from pyspark import SparkContext

In [2]:
for j in range(1,5):
    sc= SparkContext(master = "local[%d]"%(j))
    t0=time()
    for i in range(10):
        sc.parallelize([1,2]*10000).reduce(lambda x,y:x+y)
    print(f"{j} executors, time = {time()-t0}")
    sc.stop()

1 executors, time = 4.308391571044922
2 executors, time = 2.318211793899536
3 executors, time = 2.5603320598602295
4 executors, time = 2.663661003112793


#### We observe that it takes almost double time for 1 worker, and after that time reduces to a flat level for 2,3,4 workers etc. This is because this code run on a Linux virtual box using only 2 cores from the host machine. If you run this code on a machine with 4 cores, you will see benefit upto 4 cores and then the flattening out of the time taken. It also become clear that using more than one worker per core is not beneficial as it just does context-switching in that case and does not speed up the parallel computation.

## Showing the essence of _lazy_ evaluation
![](https://qph.fs.quoracdn.net/main-qimg-d49dcf35ecb7eecfc6e5b39493a0e086-c)

In [3]:
sc = SparkContext(master="local[2]")

### Make a RDD with 1 million elements

In [19]:
%%time
rdd1 = sc.parallelize(range(1000000))

CPU times: user 316 µs, sys: 5.13 ms, total: 5.45 ms
Wall time: 24.6 ms


### Some computing function - `taketime`

In [20]:
from math import cos
def taketime(x):
    [cos(j) for j in range(100)]
    return cos(x)

### Check how much time is taken by `taketime` function

In [25]:
%%time
taketime(2)

CPU times: user 21 µs, sys: 7 µs, total: 28 µs
Wall time: 31.5 µs


-0.4161468365471424

### Now do the `map` operation on the function

In [26]:
%%time
interim = rdd1.map(lambda x: taketime(x))

CPU times: user 23 µs, sys: 8 µs, total: 31 µs
Wall time: 34.8 µs


#### How come each taketime function takes 45.8 us but the map operation with a 10000 element RDD also took similar time?<br><br>Because of _lazy_ evaluation i.e. nothing was computed in the previous step, just a plan of execution was made. The variable `interim` does not point to a data structure, instead it points to a plan of execution, expressed as a dependency graph. The dependency graph defines how RDDs are computed from each other.

### Let's see the "Dependency Graph" using `toDebugString` method

In [27]:
print(interim.toDebugString().decode())

(2) PythonRDD[10] at RDD at PythonRDD.scala:49 []
 |  ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:184 []


![](https://raw.githubusercontent.com/tirthajyoti/Spark-with-Python/master/Images/RDD_dependency_graph.PNG)

### The actual execution by `reduce` method

In [28]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.28870546796843666
CPU times: user 11.6 ms, sys: 5.56 ms, total: 17.2 ms
Wall time: 15.6 s


In [29]:
1000000*31e-6

31.0

#### It is less than what we would have expected considering 1 million operations with the `taketime` function. This is the result of parallel operation of 2 cores.

### Now, we have not saved (materialized) any intermediate results in `interim`, so another simple operation (e.g. counting elements > 0) will take almost same time

In [31]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 10.6 ms, sys: 8.55 ms, total: 19.2 ms
Wall time: 12.1 s


## Caching to reduce computation time on similar operation (spending memory)

### Run the same computation as before with `cache` method to tell the dependency graph to plan for caching

In [32]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

CPU times: user 7.22 ms, sys: 4.29 ms, total: 11.5 ms
Wall time: 63 ms


In [33]:
print(interim.toDebugString().decode())

(2) PythonRDD[14] at RDD at PythonRDD.scala:49 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:184 [Memory Serialized 1x Replicated]


In [34]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.28870546796843666
CPU times: user 16.4 ms, sys: 2.24 ms, total: 18.7 ms
Wall time: 15.3 s


### Now run the same `filter` method with the help of cached result

In [35]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 14.2 ms, sys: 3.27 ms, total: 17.4 ms
Wall time: 811 ms


#### This time it took much shorter time due to cached result, which it could use to compare to 0 and count easily.